# OpenStack API

El API de OpenStack permite la gestión de todos los componentes de la infraestructura.

Vamos a realizar las siguientes operaciones como ejemplo de uso:
* autenticación
* creación de red
* creación de router
* creación de máquina virtual

## Variables de entorno
Inicializamos las variables de entorno tomando como partida el archivo de openrc.sh que nos bajamos desde la interfaz de OpenStack:

In [2]:
config={}
config["OS_AUTH_URL"]="http://openstack-vcenter:5000/v3"
config["OS_PROJECT_ID"]="9d7812704e104a208603c5d0481bd952"
config["OS_PROJECT_NAME"]="admin"
config["OS_USER_DOMAIN_NAME"]="default"
config["OS_USERNAME"]="admin"
config["OS_PASSWORD"]="admin"
config["OS_REGION_NAME"]="RegionOne"
config["name"]="prueba"

hola


## Autenticación
Enviamos nuestras credenciales y obtenemos un token en las cabeceras.

In [ ]:
# Peticiones http
import request
# Tratamiento de json
import json

    headers = {}
    headers["Content-Type"] = 'application/json'

    data = """
    {
        "auth": {
            "identity": {
                "methods": [
                    "password"
                ],
                "password": {
                    "user": {
                        "name": "%s",
                        "password": "%s",
                        "domain": {
                            "name": "%s"
                        }
                    }
                }
            },
            "scope": {
                "project": {
                    "id": "%s",
                    "domain": {
                        "name": "%s"
                    }
                }
            }
        }
    }
    """ % (config["OS_USERNAME"],
           config["OS_PASSWORD"],
           config["OS_USER_DOMAIN_NAME"],
           config["OS_PROJECT_ID"],
           config["OS_USER_DOMAIN_NAME"])
    #print data
    headers["Content-Type"] = 'application/json'
    #
    r = requests.post(config["OS_AUTH_URL"] + "/auth/tokens",
                      data=data, headers=headers)
    token = json.loads(r.text)
    token_id = r.headers["X-Subject-Token"]
    #print json.dumps(token, indent=4)
    #print (token_id)

    return (token, token_id)

